# Applied AI: Deep Learning

Key components of cloud architecture for applied AI:
* Object Storage - for capacity, redundancy, automated backup and I/O performance
* Real-Time Data Stream - for real-time ingestion and model applications
* Scaling - how to scale models on GPU and CPU clusters
* Jupyter Notebooks - to create models
* Deep Learning Framework - high or low level options to implement/test neural networks (ex: Keras)
* Open Neural Network Exchange Formats - to facilitate import/export between frameworks (ex: ONNX)
* Execution Environment - for large scale parallel execution (ex: DeepLearning4J or Apache SystemML on top of Apache Spark)

Popular deep learning frameworks include Keras, CNTK and Theano (depricated). Keras, the de facto choice, uses TensorFlow as the execution engine and can export models to be ingested by other frameworks such as DeepLearning4J and Apache System ML via open standard exchange formats such as ONNX.

### Neural Networks

Why neural networks? Linear machine learning models are limited to linear functions; on the other hand, neural networks can be used when data isn't linearly separable. 

#### Deep Feedforward Neural Networks

The simplest type of neural network is called a **perceptron**, which consists of a linear combination of an input vector and a weights vector, passed into a step activation function. This system acts as a binary linear classifier and is used to approximate some function. Deep feedforward neural networks consist of mutilayer perceptrons, with information flowing in the forward direction, through a hidden layer, in order to calculate the function output. 

Deep feedforward neural networks can represent any mathematical function (the Universal Function Approximation Theory). However, even if you can represent any mathetmatical function, having a single hidden layer is not viable for training the network.

#### Convolutional Neural Networks

Convolutional neural networks are favored for image classification due to their lower computational cost and ability to capture pixel dependencies throughout the image. Deep feedforward neural networks can represent any mathematical function.

#### Recurrent Neural Networks

While deep feedforward networks are effective at learning functions, they do not work well with sequences or time series data - enter RNNs. In these networks, feedback connections between neurons pass back temporal information, giving the system a form of memory.

#### Long Short Term Networks